# Curso Práctico de SQL y MySQL

Instructor: Alberto Alcocer


### Preparando el repositorio remoto

En GitHUb creo un nuevo repositorio con el mismo nombre: *Practico_sql*.

Una vez creado el repositorio, solo contendra un archivo readme. Ahora vamos a copiar la direccion del repositorio, dando click en el boton verde, nos aseguramos que este seleccionada con la opcion SSH, y volvemos a nuestra consola local.

Nos aseguramos que estemos en la rama master de nuestro repositorio, y vamos a decirle a git que vamos a elegir un origen remoto de nuestros archivos, con el comando:

    sudo git remote add origin [direccion del repo]

para nuestro repositorio especificio seria:

    sudo git remote add origin git@github.com:carlcri/Practico_sql.git

Verificamos con:

    git remote -v


Vamos a preparar nuestro repositorio antes de hacer el PUSH para llevar los archivos desde el repo local al remoto.

Asi que lo primero es hacer un PULL para integrar los cambios remotos, esta sera una operacion especial, con este comando:

    sudo git pull origin main --allow-unrelated-histories

Analizando este comando con cuidado vemos que el *origin* es *main*, debido a que en GitHub por defecto, no existe la rama MASTER sino MAIN

Y al ejecutarlo da un error bastante molesto: 😭😭

![](https://i.imgur.com/2BaW7eM.png)

¿que hacer?

vamos a intentar

     git config pull.rebase false 

y volvemos a intentar. El comando anterior. Y funciono 😇

### Recuerda puedes revisar la configuracion asi:

    git config --list


Una vez se ejecuta el anterior comando, entramos a una pantalla que siempre es dificil de cerrar, por lo general se cierra con una combinaciond de teclas como son:

    CTRL + X
    CTRL + X + X

Al revisar la carpeta del repositorio con el comando *ls* se observa hay un archivo nuevo que antes no estaba, peor si en el remoto: *README.md*.

Ahora estamos listos para el PULL, ¡pero antes!

### ¿que pasara si desactivo vuelvo a colocar el rebase como estaba?

    git config pull.rebase true

Y realizamos el PULL

    sudo git pull origin main

![](https://i.imgur.com/zeAD2nP.png)


Parece que todo salio bien!

Y ahora si el famoso *push*

    sudo git push origin master:main

Y perfecto ✅

Revisamos lo que hay dentro del repositorio en GITHUB y se observa los cambios locales se guardaron en el remoto. Asi que demonos por bien servidos.

### Configurar Cuenta Alterna como Colaborador

Mi cuenta alterna en GitHub *gataapestosa* hay que añadirla como colaborador, porque ella, desde su repositorio local, sera quien realize las practicas y contenidos del curso.

Desde los *settings* del repositorio, ir a la parte de collaborators y añadirla.

Delegar a *gataapestosa* para que cree las ramas, y las suba a *Github*

#### Clonar el Proyecto

El entorno de trabajo de la cuenta alterna es:

    ~/warehouse/anita

Desde esta ubicacion, clonar el proyecto, (Boton Verde para obtener la ubicacion https), como se configuro publico, no habra ningun problema en este paso, :

     git clone https://github.com/carlcri/Practico_sql.git

### Ramas

Deben ser creadas por la cuenta alterna

En el entorno local habra dos ramas, una principal y otra de desarrollo. El objetivo es practicar los comandos de *git*.

1.  master
2.  development

Solo se trabajara en la rama development, y al final se realizara un merge a la rama principal. Para crear la rama

    git branch development

#### comandos utiles para trabajar con ramas

    git branch
    git show-branch --all

    git log 
    git log -all
    git log --graph
    git log --graph --decorate --oneline
    git log --all --graph --decorate --oneline

## Realizando cambios en la rama development

Asegurarse estar trabajando desde el *entorno de trabajo de la cuenta alterna* el branch *development*. Igualmente, como se ha clonado el proyecto, verificar con el comando *git remote*

![origin](https://i.imgur.com/E78OjTs.png)

Observamos el origen esta vinculado al protocolo https y no SSh

Guardamos los cambios con un *commit* y procedemos hacer un *push* usando la cuenta alterna. El username sera el de la cuenta alterna, y el password, el token generado y guardado en un archivo de texto plano 

![push](https://i.imgur.com/6ihxPgR.png)

## Docker

### Docker Hub and Image

Ir a docker hub y traer un nueva imagen de *mysql*, sugiere usar: 

    sudo docker pull mysql:latest 

Y para verificar que se hay una nueva imagen con el tag *latest*:

    sudo docker images

![](https://i.imgur.com/tDMRtpu.png)

### Primeros Pasos

Este tutorial fue muy util: https://platzi.com/tutoriales/1432-docker-2018/3268-como-crear-un-contenedor-con-docker-mysql-y-persistir-la-informacion/

Ejecutamos los comando sugeridos en la documentacion:

    docker run --name mysql-db -e MYSQL_ROOT_PASSWORD=mysecret -d mysql

Donde mysql-db es el nombre del contenedor, y *mysecret* is the password to be set for the MySQL root user.

Comandos utiles:

    docker ps -a
    docker rm
    docker rm -f

#### Entrando al contenedor

Para entrar al contenedor usamos un modo interactivo para asignar un TTY(terminal) 

    docker exec -it mysql-db mysql -p

Donde **mysql -p**: es el comando para entrar a la consola de mysql con el usuario root. 

Una vez demos el password, estaremos dentro del contenedor y lanzar comandos a MYSQL

![](https://i.imgur.com/idnpfKB.png)

Por ejemplo, creemos una base de datos que se llame mascotas y una tabla:

    create database if not exists Mascotas;
    use Mascotas;
    create table if not exists perros(id INT);

Sin embargo ahora mismo no es posible persisitir los datos. Al parar el contenedor los datos se perderan. 😞

### Montar un Volumen

Hacer que los datos persistan

Parar el container y eliminarlo:

    docker rm -f mysql-db 

Eliminamos todos los volúmenes ya que Docker crea volúmenes temporales sin pedirte permiso.

    docker volume prune
    docker volume ls

Crearemos un nuevo volumen:

    docker volume create mysql-db-data

Y recreamos el container agregando el volumen con la opcion mount, cualquiera de las dos opciones sirve:


    docker run --name mysql-db -e MYSQL_ROOT_PASSWORD=mysecret -d --mount src=mysql-db-data,dst=/var/lib/mysql mysql

    docker run --name mysql-db --mount src=mysql-db-data,dst=/var/lib/mysql -e MYSQL_ROOT_PASSWORD=mysecret -d mysql

🔴 Pero hay que tener cuidado, porque en el primero se asigna primero el password, y despues se monta el volumen. Al parecer esto hace que se ignore el statement *MYSQL_ROOT_PASSWORD=mysecret*, y guade por defecto algun password almacenado en el volumen. Asi que cuidado

Y volvemos a ingresar:

    docker exec -it mysql-db mysql -p

- Creamos una la bd mascotas con su tabla perros.
- Removemos el container, volvmes a crearlo, y volvemos a ingresar:


        docker rm -f mysql-db

        docker run --name mysql-db --mount src=mysql-db-data,dst=/var/lib/mysql -e MYSQL_ROOT_PASSWORD=mysecret -d mysql
    
        docker exec -it mysql-db mysql -p

![](https://i.imgur.com/oOUJWay.png)

Y acabamos de verificar nuestra base de datos *mascotas* existe.


## Instalando Notepad

Instale *Notepadqq*. Para lanzarlo abrir una consola y:

    notepadqq